In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

print("TensorFlow Version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))


TensorFlow Version: 2.17.0
GPUs: []


In [2]:
DATASET_PATH = "../data/raw/food-101/"

IMG_DIR = os.path.join(DATASET_PATH, "images")
META_DIR = os.path.join(DATASET_PATH, "meta")

train_file = os.path.join(META_DIR, "train.txt")
test_file = os.path.join(META_DIR, "test.txt")

print("Images:", IMG_DIR)
print("Train file:", train_file)
print("Test file:", test_file)


Images: ../data/raw/food-101/images
Train file: ../data/raw/food-101/meta/train.txt
Test file: ../data/raw/food-101/meta/test.txt


In [3]:
def load_split(file_path):
    with open(file_path, "r") as f:
        items = [line.strip() for line in f.readlines()]
    return items

train_list = load_split(train_file)
test_list = load_split(test_file)

print("Train items:", len(train_list))
print("Test items:", len(test_list))


Train items: 75750
Test items: 25250


In [4]:
def add_ext(x):
    return x + ".jpg"

train_paths = [add_ext(x) for x in train_list]
test_paths = [add_ext(x) for x in test_list]


In [5]:
train_df = pd.DataFrame({
    "filename": train_paths,
    "class": [x.split("/")[0] for x in train_list]
})

test_df = pd.DataFrame({
    "filename": test_paths,
    "class": [x.split("/")[0] for x in test_list]
})

print(train_df.head())


                filename      class
0  apple_pie/1005649.jpg  apple_pie
1  apple_pie/1014775.jpg  apple_pie
2  apple_pie/1026328.jpg  apple_pie
3  apple_pie/1028787.jpg  apple_pie
4  apple_pie/1043283.jpg  apple_pie


In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [7]:
train_gen = train_datagen.flow_from_dataframe(
    train_df,
    directory=IMG_DIR,
    x_col="filename",
    y_col="class",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = test_datagen.flow_from_dataframe(
    test_df,
    directory=IMG_DIR,
    x_col="filename",
    y_col="class",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False
)

num_classes = len(train_gen.class_indices)

print("Total classes:", num_classes)
print("Class index sample:", list(train_gen.class_indices.items())[:10])


Found 75750 validated image filenames belonging to 101 classes.
Found 25250 validated image filenames belonging to 101 classes.
Total classes: 101
Class index sample: [('apple_pie', 0), ('baby_back_ribs', 1), ('baklava', 2), ('beef_carpaccio', 3), ('beef_tartare', 4), ('beet_salad', 5), ('beignets', 6), ('bibimbap', 7), ('bread_pudding', 8), ('breakfast_burrito', 9)]


In [8]:
base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base.trainable = False   # IMPORTANT


In [9]:
inputs = base.input
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs, outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [10]:
model.compile(
    optimizer=Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [11]:
EPOCHS_HEAD = 5

history_head = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_HEAD
)

model.save("../data/clean/effb0_head.h5")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
1380/2368 ━━━━━━━━━━━━━━━━━━━━ 8:15 502ms/step - accuracy: 0.0104 - loss: 4.7139

KeyboardInterrupt: 

In [ ]:
base.trainable = True  # UNFREEZE EVERYTHING


In [ ]:
for layer in base.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = False


In [ ]:
model.compile(
    optimizer=Adam(1e-5),   # VERY IMPORTANT
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
EPOCHS_FT = 15

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_FT
)

model.save("../data/clean/effb0_finetuned.h5")


In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history_head.history["accuracy"] + history_ft.history["accuracy"])
plt.plot(history_head.history["val_accuracy"] + history_ft.history["val_accuracy"])
plt.title("Training + Fine-Tuning Accuracy")
plt.legend(["train","val"])

plt.subplot(1,2,2)
plt.plot(history_head.history["loss"] + history_ft.history["loss"])
plt.plot(history_head.history["val_loss"] + history_ft.history["val_loss"])
plt.title("Training + Fine-Tuning Loss")
plt.legend(["train","val"])

plt.show()
